In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
from tqdm import tqdm
# from model.bert_things.pytorch_pretrained_bert.tokenization import BertTokenizer
# from model.bert_things.pytorch_pretrained_bert import BertConfig, BertModel, BertPreTrainedModel
# from model.bert_text_model import BertTextModel
from data_loader.utils.vocab import Vocab
from data_loader.data_loaders import ClassificationDataLoader
import pickle
import sys
import logging
import numpy as np
import argparse
import torch
import re
from data_loader.seqcode_dataset import SeqCodeDataset

import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
from train import get_instance
from train import import_module
from model.metric import pr_auc_1, roc_auc_1

/Users/chungyan.cheung/anaconda3/envs/taper/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [4]:
# transformer_state_path = "saved/red code/1113_143730/model_best.pth"
textcode_state_path = "saved/textcode diag proc/1110_112247/model_best.pth"
state_dict = torch.load(textcode_state_path)

In [5]:
state_dict.keys()

dict_keys(['arch', 'epoch', 'logger', 'state_dict', 'optimizer', 'monitor_best', 'config'])

In [8]:
state_dict['monitor_best']

0.12825752413550087

In [15]:
state_dict['state_dict']['word_emb.embedding_w'].size()

torch.Size([384, 884])

In [7]:
state_dict['config']

{'name': 'textcode diag proc',
 'n_gpu': 1,
 'model': {'type': 'MemTransformerLM',
  'module_name': 'mem_transformer',
  'init': 'init',
  'init_range': 0.1,
  'init_std': 0.02,
  'proj_init_std': 0.01,
  'args': {'n_token': 884,
   'n_layer': 2,
   'd_model': 384,
   'n_head': 8,
   'd_head': 64,
   'd_inner': 2048,
   'dropout': 0.0,
   'tie_weight': False,
   'd_embed': 384,
   'dropatt': 0.0,
   'tgt_len': 0,
   'mem_len': 0,
   'ext_len': 0,
   'demographics_len': 0}},
 'data_loader': {'type': 'SeqCodeDataLoader',
  'args': {'data_dir': '~/mimic_taper',
   'batch_size': 32,
   'shuffle': True,
   'validation_split': 0.1,
   'num_workers': 1,
   'med': False,
   'cptcode': True,
   'proc': False,
   'diag': True}},
 'optimizer': {'type': 'Adam',
  'args': {'lr': 0.00015, 'weight_decay': 0, 'amsgrad': True}},
 'loss': 'med2vec_loss_transformer',
 'clip_grad': 0.25,
 'loss_window': 2,
 'metrics': ['recall_10',
  'recall_20',
  'recall_30',
  'recall_40',
  'recall_10_proc',
  'recall

In [22]:
config = state_dict['config']

In [19]:
vocab_path = 'saved/red code/predictions.pkl'
predictions = pickle.load(open(vocab_path, 'rb'))
predictions['target'].size

1707

In [31]:
predictions['target'].sum()

194

In [33]:
metric_fns = [getattr(module_metric, met) for met in config['metrics']]
metric_fns

[<function model.metric.accuracy2(output, target, t=0)>,
 <function model.metric.sensitivity(output, target, t=0.5)>,
 <function model.metric.specificity(output, target, t=0.5)>,
 <function model.metric.precision(output, target, t=0.5)>,
 <function model.metric.pr_auc(output, target)>,
 <function model.metric.roc_auc(output, target)>]

In [37]:
print(pr_auc_1(predictions['output'], predictions['target']))
print(roc_auc_1(predictions['output'], predictions['target']))

0.1544591909615568
0.6009123677271209


In [24]:
data_loader = get_instance(module_data, 'data_loader', config)
data_loader

In [26]:
data_loader = data_loader.split_validation()

In [27]:
len(data_loader)

54

In [4]:
seqcode = SeqCodeDataset('/Users/chungyan.cheung/mimic_taper/', 32, diag=True, cptcode=True)

In [15]:
seqcode.num_codes

884

In [16]:
seqcode.num_dcodes

232

In [17]:
seqcode.num_ccodes

652

In [34]:
seqcode[seqcode.keys[4]]

IndexError: index 1111 is out of bounds for dimension 0 with size 884

In [40]:
seq = seqcode.data[seqcode.keys[4]]

In [41]:
len(seq)

3

In [42]:
s = seq[0]

In [105]:
x = torch.zeros((seqcode.num_codes, seqcode.max_len ), dtype=torch.long)
d = torch.zeros((seqcode.demographics_shape, seqcode.max_len), dtype=torch.float)
mask = torch.zeros((seqcode.max_len, ), dtype=torch.float)
ivec = []
jvec = []

In [106]:
dcode = list(s['diagnoses']) * seqcode.diag
pcode = list(len(seqcode.diag_vocab) * seqcode.diag + np.asarray(s['procedures'])) * seqcode.proc
mcode = list(len(seqcode.diag_vocab) * seqcode.diag + len(seqcode.proc_vocab) * seqcode.proc + np.asarray(s['medications'])) * seqcode.med
cptcode = list(len(seqcode.diag_vocab) * seqcode.diag + np.asarray(s['cptproc'])) * seqcode.cpt
demo = s['demographics']
ss = (dcode) + (pcode) + (mcode) + (cptcode)
ss

[79, 105, 79, 220, 78, 95, 128, 28, 459.0, 459.0, 459.0, 463.0, 463.0]

In [107]:
i = 0
for j in ss:
    for k in ss:
        if j == k:
            continue
    ivec.append(j)#torch.cat(ivec, i)
    jvec.append(k)#torch.cat(jvec, j)
x[ss, i] = 1
d[:, i] = torch.Tensor(demo)
mask[i] = 1

In [108]:
print(ivec)
print(jvec)
x.size()

[79, 105, 79, 220, 78, 95, 128, 28, 459.0, 459.0, 459.0, 463.0, 463.0]
[463.0, 463.0, 463.0, 463.0, 463.0, 463.0, 463.0, 463.0, 463.0, 463.0, 463.0, 463.0, 463.0]


torch.Size([884, 40])

In [110]:
len(ivec)

13

In [116]:
seqcode.data[seqcode.keys[1]][0]['cptproc']
mask

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [82]:
seqcode.data_info

{'num_patients': 38402,
 'num_icd9_codes': 3444,
 'num_proc_codes': 1052,
 'num_med_codes': 3400,
 'demographics_shape': 91,
 'demographic_cols': {'AGE': [],
  'GENDER': Index(['M', 'F'], dtype='object'),
  'LAST_CAREUNIT': Index(['SICU', 'CSRU', 'MICU', 'TSICU', 'CCU', 'NICU'], dtype='object'),
  'MARITAL_STATUS': Index(['MARRIED', 'SINGLE', 'DIVORCED', 'WIDOWED', 'UNKNOWN (DEFAULT)',
         'LIFE PARTNER', 'SEPARATED'],
        dtype='object'),
  'ETHNICITY': Index(['WHITE', 'BLACK/AFRICAN AMERICAN', 'ASIAN', 'OTHER',
         'HISPANIC OR LATINO', 'UNKNOWN/NOT SPECIFIED',
         'PATIENT DECLINED TO ANSWER', 'UNABLE TO OBTAIN', 'BLACK/CAPE VERDEAN',
         'WHITE - RUSSIAN', 'ASIAN - CHINESE', 'MULTI RACE ETHNICITY',
         'HISPANIC/LATINO - PUERTO RICAN', 'PORTUGUESE',
         'HISPANIC/LATINO - DOMINICAN', 'ASIAN - VIETNAMESE', 'ASIAN - FILIPINO',
         'HISPANIC/LATINO - COLOMBIAN', 'AMERICAN INDIAN/ALASKA NATIVE',
         'HISPANIC/LATINO - GUATEMALAN', 'ASIAN - AS

In [12]:
data['data'][3]

[{'demographics': [76,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'diagnoses': [77, 79, 79, 122, 1, 46, 79, 199, 158],
  'procedures': [0, 1, 2, 3, 4, 5],
  'medications': [],
  'cptproc': [231.0, 231.0, 231.0, 231.0],
  'timedelta': 0,
  'los': 6,
  'readmission': 0.0,
  'mortality': False}]

In [166]:
cptevents_path = '/Users/chungyan.cheung/mimic-iii-clinical-database-1.4/CPTEVENTS.csv'
cptevents = pd.read_csv(cptevents_path, dtype={'CPT_CD': str})[['CPT_CD']].drop_duplicates()
cptevents.head()

/Users/chungyan.cheung/anaconda3/envs/taper/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CPT_CD
0,99232
4,99223
6,99233
8,99231
10,99238


In [156]:
def process_cr(cr):
    t = cr.split('-')
    mi, ma = t[0].strip().strip("'"), t[1].strip().strip("'")
    return [mi, ma]

In [158]:
ccs_path = '/Users/chungyan.cheung/HCUP-US-EHR-maps/2019_ccs_services_procedures.csv'
ccs_map = pd.read_csv(ccs_path, header=1)
ccs_map['code_start'] = ccs_map['Code Range'].apply(lambda x: process_cr(x)[0])
ccs_map['code_end'] = ccs_map['Code Range'].apply(lambda x: process_cr(x)[1])
ccs.head()

,Code Range,CCS,CCS Label,code_start,code_end
0,'61000-61055',1,Incision and excision of CNS,61000,61055
1,'61105-61106',1,Incision and excision of CNS,61105,61106
2,'61108-61130',1,Incision and excision of CNS,61108,61130
3,'61150-61156',1,Incision and excision of CNS,61150,61156
4,'61250-61315',1,Incision and excision of CNS,61250,61315


In [189]:
def map_ccs(code):
    ccs_group = ccs_map.query(f'(code_start <= "{code}") & (code_end >= "{code}")').CCS
    if len(ccs_group) == 0:
        return None
    elif len(ccs_group) > 1:
        raise ValueError(f'Invlaid code: {code}')
    else:
        return ccs_group.iloc[0]

cptevents['CCS'] = cptevents['CPT_CD'].apply(map_ccs)


In [199]:
{x:y for x,y in cptevents.values}

{'99232': 227.0,
 '99223': 227.0,
 '99233': 227.0,
 '99231': 227.0,
 '99238': 227.0,
 '99239': 227.0,
 '99254': 227.0,
 '99255': 227.0,
 '99222': 227.0,
 '32000': 39.0,
 '99220': 227.0,
 '99291': 227.0,
 '62272': 4.0,
 '31629': 37.0,
 '62270': 4.0,
 '99253': 227.0,
 '43752': 221.0,
 '36489': 54.0,
 '99251': 227.0,
 '99236': 227.0,
 '99292': 227.0,
 '49080': 88.0,
 '93503': 204.0,
 '99235': 227.0,
 '99252': 227.0,
 '99262': 227.0,
 '99263': 227.0,
 '99219': 227.0,
 '99261': 227.0,
 '82803': 205.0,
 '33210': 48.0,
 '49081': 88.0,
 '99221': 227.0,
 '20610': 155.0,
 '92960': 63.0,
 '99234': 227.0,
 '90935': 58.0,
 '90937': 58.0,
 '99218': 227.0,
 '93743': 203.0,
 '99358': 227.0,
 '99356': 227.0,
 '36415': 231.0,
 '93561': 47.0,
 '90945': 91.0,
 '43235': 70.0,
 '99141': 231.0,
 '99357': 227.0,
 '36620': 54.0,
 '90947': 91.0,
 '93744': 203.0,
 '93742': 203.0,
 '36800': 57.0,
 '50200': 110.0,
 '43205': 70.0,
 '33211': 48.0,
 '20600': 155.0,
 '36600': 231.0,
 '45331': 77.0,
 '38220': 65.0,
 '3

In [193]:
ccs_map.query('CCS == 108')

,Code Range,CCS,CCS Label,code_start,code_end
1270,'51701-51703',108,Indwelling catheter,51701,51703
1271,'53670-53675',108,Indwelling catheter,53670,53675
1272,'G0002-G0002',108,Indwelling catheter,G0002,G0002


In [2]:
cpt_vocab = Vocab()

In [202]:
vocab_path = '/Users/chungyan.cheung/HCUP-US-EHR-maps/cpt.vocab'
cpt_dict = pickle.load(open(vocab_path, 'rb'))

In [203]:
cpt_

NameError: name 'cpt_' is not defined

In [21]:
cpt_df = cpt_df.astype({'min': str, 'max': str})
cpt_df['min'] = cpt_df['min'].str.pad(5, 'right', '0')
cpt_df['max'] = cpt_df['max'].str.pad(5, 'right', '0')

In [26]:
cpt_code = '22305'
cpt_df.query(f'min <= "{cpt_code}" & max >= "{cpt_code}"')

,CCS,CCS Label,min,max,minc,maxc
1777,159,Other diagnostic procedures on musculoskeletal...,22000,22510,,
2427,202,Electrocardiogram,22300,22500,T,T


In [52]:
vocab_path2 = '/Users/chungyan.cheung/HCUP-US-EHR/example_output/cpt.vocab'
cptvocab = pickle.load(open(vocab_path2, 'rb'))
len(cptvocab)

1989

In [15]:
cpt_vocab._build_from_file(vocab_path)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [196]:
cpt_vocab.sym2idx['12007']

171.0